# Data Loading, Cleaning & Export

### Importazione Librerie

In [17]:
import numpy as np
import pandas as pd

### Caricamento Dati

In [32]:
CSV_files = {
    "customers":           r"dataset/olist_customers_dataset.csv",
    "geolocation_dataset": r"dataset/olist_geolocation_dataset.csv",
    "order_items":         r"dataset/olist_order_items_dataset.csv",
    "order_payments":      r"dataset/olist_order_payments_dataset.csv",
    "order_review":        r"dataset/olist_order_reviews_dataset.csv",
    "order_dataset":       r"dataset/olist_orders_dataset.csv",
    "list_product":        r"dataset/olist_products_dataset.csv",
    "list_seller":         r"dataset/olist_sellers_dataset.csv",
    "product_category":    r"dataset/product_category_name_translation.csv"
}

dataframes = {name: pd.read_csv(path) for name, path in CSV_files.items()}

df_customers           = dataframes["customers"]
df_orders_items        = dataframes["order_items"]
df_order_payments      = dataframes["order_payments"]
df_order_review        = dataframes["order_review"]
df_order_dataset       = dataframes["order_dataset"]
df_list_product        = dataframes["list_product"]
df_list_seller         = dataframes["list_seller"]
df_product_category    = dataframes["product_category"]
df_geolocation_dataset = dataframes["geolocation_dataset"]

### Analisi Preliminare

In [33]:
# Stampa un riepilogo di ogni csv con la sua shape, i missing values e i duplicati.
def quick_overview(df, name):

    # Nome DF
    print(f"{name}")
    print(f"Shape: {df.shape}")

    # Valori nulli
    nulls = df.isna().sum()
    nulls = nulls[nulls > 0]
    if len(nulls) > 0:
        print(f"Valori nulli:\n{nulls.to_string()}\n")
    else:
        print("Nessun valore nullo\n")

for name, df in dataframes.items():
    quick_overview(df, name)

customers
Shape: (99441, 5)
Nessun valore nullo

geolocation_dataset
Shape: (1000163, 5)
Nessun valore nullo

order_items
Shape: (112650, 7)
Nessun valore nullo

order_payments
Shape: (103886, 5)
Nessun valore nullo

order_review
Shape: (99224, 7)
Valori nulli:
review_comment_title      87656
review_comment_message    58247

order_dataset
Shape: (99441, 8)
Valori nulli:
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965

list_product
Shape: (32951, 9)
Valori nulli:
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2

list_seller
Shape: (3095, 4)
Nessun valore nullo

product_category
Shape: (71, 2)
Nessun valore nullo



### PULIZIA: df_list_product

In [34]:
# 1) Droppiamo le colonne (peso, misure)
# 2) Droppiamo i 610 prodotti senza categoria (< 2% del totale):
#    verranno esclusi anche dagli altri df.
# 3) Uniamo la traduzione inglese della categoria.

# 1
df_list_product = df_list_product.drop(
    columns=['product_name_lenght', 'product_width_cm',
             'product_height_cm', 'product_length_cm', 'product_weight_g']
)

# 2
# Salviamo gli id dei prodotti senza categoria per filtrare order_items dopo
product_id_nan = df_list_product[df_list_product['product_category_name'].isna()]
product_id_to_delete = product_id_nan['product_id'].to_numpy()
df_list_product = df_list_product.dropna(subset=['product_category_name'])

# 3
df_list_product = (
    df_list_product
    .merge(df_product_category, on='product_category_name', how='left')
)

df_list_product.info()
df_list_product.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 32341 entries, 0 to 32340
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32341 non-null  str    
 1   product_category_name          32341 non-null  str    
 2   product_description_lenght     32341 non-null  float64
 3   product_photos_qty             32341 non-null  float64
 4   product_category_name_english  32328 non-null  str    
dtypes: float64(2), str(3)
memory usage: 1.2 MB


,product_id,product_category_name,product_description_lenght,product_photos_qty,product_category_name_english
3629,6eefea74d72822573892405c674918e6,cool_stuff,1114.0,5.0,cool_stuff
18932,1600dcf1cea8c0c83702e07b577ab231,relogios_presentes,181.0,4.0,watches_gifts


In [38]:
# Traduciamo con Selenium le categorie portoghesi senza traduzione inglese
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Troviamo i nomi portoghesi che non hanno traduzione nel dataframe df_product_category
traduzione_mancante = df_list_product['product_category_name_english'].isna()
nomi_mancanti = df_list_product[traduzione_mancante]['product_category_name'].unique()
print(f"Categorie da tradurre ({len(nomi_mancanti)}): \n{nomi_mancanti}\n")


Categorie da tradurre (2): 
<StringArray>
['pc_gamer', 'portateis_cozinha_e_preparadores_de_alimentos']
Length: 2, dtype: str



In [45]:
# Apre Google Translate nel browser e restituisce la traduzione
# dal portoghese all'inglese del testo passato, formattata in snake_case.
def traduci_categoria(testo):

    # Costruisce l'URL di Google Translate con i parametri: lingua portoghese (pt), lingua inglese (en) e testo
    url = f"https://translate.google.com/?sl=pt&tl=en&text={testo}&op=translate"
    driver.get(url)
    
    # Attende 2 secondi che la pagina carichi e la traduzione appaia
    time.sleep(2)

    # Accetta i cookie se il popup è presente
    try:
        accept_button = driver.find_element(By.XPATH, "//button[.//span[text()='Accept all']]")
        accept_button.click()
        time.sleep(2)
    except:
        pass  # Il popup non è presente, continua normalmente
    
    # Trova l'elemento HTML che contiene il testo tradotto
    risultato = driver.find_element(By.XPATH, "//span[@jsname='W297wb']")
    
    # Converte in minuscolo e sostituisce gli spazi con underscore
    traduzione = risultato.text.lower().replace(' ', '_')
    return traduzione

# Avvia Firefox
# NB CONTROLLATE TUTTI SE SI APRE IL BROWSER E VEDETE GOOGLE TRANSLATE
driver = webdriver.Firefox()

# Dizionario che conterrà le traduzioni: {nome_portoghese: traduzione_inglese}
traduzioni_manuali = {}

# Itera sui nomi di categoria mancanti di traduzione
for nome in nomi_mancanti:
    traduzione = traduci_categoria(nome)
    traduzioni_manuali[nome] = traduzione
    print(f"{nome}  →  {traduzione}")
    time.sleep(1)

# Chiude il browser al termine delle traduzioni
driver.quit()

# Aggiorna il DataFrame: per ogni categoria tradotta,
# riempie i valori NaN nella colonna 'product_category_name_english'
for nome, traduzione in traduzioni_manuali.items():
    df_list_product.loc[
        df_list_product['product_category_name'] == nome,
        'product_category_name_english'
    ] = traduzione

df_list_product.info()

pc_gamer  →  gaming_pc
portateis_cozinha_e_preparadores_de_alimentos  →  portateis_kitchen_e_food_preparators
<class 'pandas.DataFrame'>
RangeIndex: 32341 entries, 0 to 32340
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32341 non-null  str    
 1   product_category_name          32341 non-null  str    
 2   product_description_lenght     32341 non-null  float64
 3   product_photos_qty             32341 non-null  float64
 4   product_category_name_english  32341 non-null  str    
dtypes: float64(2), str(3)
memory usage: 1.2 MB


,product_id,product_category_name,product_description_lenght,product_photos_qty,product_category_name_english
17685,a48138f7de6ebfed36f91a440fcadb64,esporte_lazer,1696.0,1.0,sports_leisure
24027,39d111da3ecf6cae8cca1d6d2946585f,ferramentas_jardim,427.0,5.0,garden_tools


### PULIZIA: df_orders_items

In [99]:
# 1) Escludiamo gli items con product_id in product_id_to_delete
# 2) Convertiamo shipping_limit_date in datetime

# 1
df_orders_items = (
    df_orders_items[~df_orders_items['product_id'].isin(product_id_to_delete)]
    .copy()
    .reset_index(drop=True)
)

# 2
df_orders_items['shipping_limit_date'] = pd.to_datetime(
    df_orders_items['shipping_limit_date']
)

df_orders_items.info()
df_orders_items.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 111047 entries, 0 to 111046
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             111047 non-null  str           
 1   order_item_id        111047 non-null  int64         
 2   product_id           111047 non-null  str           
 3   seller_id            111047 non-null  str           
 4   shipping_limit_date  111047 non-null  datetime64[us]
 5   price                111047 non-null  float64       
 6   freight_value        111047 non-null  float64       
dtypes: datetime64[us](1), float64(2), int64(1), str(3)
memory usage: 5.9 MB


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
12398,1c8757886a51b1d2a57ef26e7004ee37,1,dd3575a8c5e2139f680a9816a15c8f2a,4869f7a5dfa277a7dca6462dcf3b52b2,2017-03-23 19:26:44,235.0,15.82
86462,c71306562d226384a996365f61966a78,1,7764215b7e09bfc5d5d1c8fe89eef727,2f74af7a0ee5636f12c2336f9fffed47,2017-10-12 21:56:12,38.9,21.15


### PULIZIA: df_order_dataset

In [100]:
# 1) Conversione delle colonne data in datetime

# 2) delivery_delay_days : differenza tra consegna effettiva e stimata
#    (positivo = in ritardo, negativo = in anticipo)
#    actual_delivery_days: giorni totali dall'acquisto alla consegna

# 1
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]
for col in date_cols:
    df_order_dataset[col] = pd.to_datetime(df_order_dataset[col])

# 2
df_order_dataset['delivery_delay_days'] = (
    df_order_dataset['order_delivered_customer_date'] -
    df_order_dataset['order_estimated_delivery_date']
).dt.days
df_order_dataset['actual_delivery_days'] = (
    df_order_dataset['order_delivered_customer_date'] -
    df_order_dataset['order_purchase_timestamp']
).dt.days

df_order_dataset.info()
df_order_dataset.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  str           
 1   customer_id                    99441 non-null  str           
 2   order_status                   99441 non-null  str           
 3   order_purchase_timestamp       99441 non-null  datetime64[us]
 4   order_approved_at              99281 non-null  datetime64[us]
 5   order_delivered_carrier_date   97658 non-null  datetime64[us]
 6   order_delivered_customer_date  96476 non-null  datetime64[us]
 7   order_estimated_delivery_date  99441 non-null  datetime64[us]
 8   delivery_delay_days            96476 non-null  float64       
 9   actual_delivery_days           96476 non-null  float64       
dtypes: datetime64[us](5), float64(2), str(3)
memory usage: 7.6 MB


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days,actual_delivery_days
33377,a68ec02f230006f4c1ed1fd0a64c23ca,f619b4330701af65eb83e1540d775550,delivered,2018-05-03 13:28:29,2018-05-03 13:52:40,2018-05-03 14:14:00,2018-05-11 18:03:11,2018-05-29,-18.0,8.0
85244,17ade9a25d129b033c406e6d31b31a85,dd7148aec251124d2e36e35ec175b66d,delivered,2017-03-03 08:55:57,2017-03-04 02:24:05,2017-03-14 10:25:43,2017-03-22 04:27:45,2017-03-24,-2.0,18.0


### PULIZIA: df_order_review

In [101]:
# 1) Conversione della colonna review_creation_date in datetime
# 2) Droppiamo le colonne (review_comment_title, review_comment_message)

# 1
df_order_review['review_creation_date'] = pd.to_datetime(
    df_order_review['review_creation_date']
)

# 2
df_order_review = df_order_review.drop(columns= ['review_comment_title', 'review_comment_message'])

df_order_review.sample(2)

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp
98151,07be637df3aa409e37b5a3679372cca6,1ddb26b7e0414dad836e1a828c8ecfde,5,2018-01-07,2018-01-08 04:20:48
41614,5af48d7c9750ca39d0f67d404492f475,ecf7e61d0f5617017f9105bf07f390f6,5,2017-02-25,2017-02-26 01:13:27


### PULIZIA: df_order_payments

In [102]:
# Per un ordine ci sono diverse righe di pagamenti
# Aggreghiamo per order_id

df_order_payments = (
    df_order_payments
    .groupby('order_id', as_index=False)
    .agg(
        total_payment_value  = ('payment_value', 'sum'),
        payment_installments = ('payment_installments', 'max'),
        payment_type         = ('payment_type', 'first')
    )
)

df_order_payments.sample(2)

,order_id,total_payment_value,payment_installments,payment_type
54105,8bd56cbba10cec53a49503a4377ce707,23.33,1,credit_card
74218,bf294e9fc4071077ff5ffbeb803602fd,41.88,1,boleto


### PULIZIA: df_geolocation_dataset

In [103]:
# Coordinate del Brasile
brazil = {
    "lat_min": -34.0, "lat_max": 5.0,
    "lng_min": -75.0, "lng_max": -28.0
}

# Filtra coordinate fuori dal Brasile
df_geolocation_clean = df_geolocation_dataset[
    df_geolocation_dataset["geolocation_lat"].between(brazil["lat_min"], brazil["lat_max"]) &
    df_geolocation_dataset["geolocation_lng"].between(brazil["lng_min"], brazil["lng_max"])
].copy()

# Raggruppa per zip code (media lat/lng, primo valore per città e stato)
df_geolocation_dataset = (
    df_geolocation_clean
    .groupby('geolocation_zip_code_prefix', as_index=False)
    .agg(
        geolocation_lat   = ('geolocation_lat',   'mean'),
        geolocation_lng   = ('geolocation_lng',   'mean'),
        geolocation_city  = ('geolocation_city',  'first'),
        geolocation_state = ('geolocation_state', 'first')
    )
)

df_geolocation_dataset.info()
df_geolocation_dataset.sample(2)

<class 'pandas.DataFrame'>
RangeIndex: 19011 entries, 0 to 19010
Data columns (total 5 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geolocation_zip_code_prefix  19011 non-null  int64  
 1   geolocation_lat              19011 non-null  float64
 2   geolocation_lng              19011 non-null  float64
 3   geolocation_city             19011 non-null  str    
 4   geolocation_state            19011 non-null  str    
dtypes: float64(2), int64(1), str(2)
memory usage: 742.7 KB


,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
6269,19273,-22.573882,-53.044630,rosana,SP
8024,29909,-19.299555,-40.066206,linhares,ES


### PULIZIA: df_list_seller

In [104]:
df_list_seller=df_list_seller.rename(columns={'seller_zip_code_prefix':'geolocation_zip_code_prefix'})
df_list_seller.sample(3)

,seller_id,geolocation_zip_code_prefix,seller_city,seller_state
1957,58e4b302b54937e55a678c4d15111da4,83402,colombo,PR
1583,b347677812ea483b0f528eaf8cbc09b7,82540,curitiba,PR
2212,a3b42d266fa8afc874b909422ce88582,14620,orlandia,SP


### PULIZIA: df_customers

In [105]:
df_customers=df_customers.rename(columns={'customer_zip_code_prefix':'geolocation_zip_code_prefix'})
df_customers.sample(3)

,customer_id,customer_unique_id,geolocation_zip_code_prefix,customer_city,customer_state
11647,eec74815d488bfc5f2c9b9a2d919ea4a,3a27aeed5e91687035a9f1c174ab7e81,15810,catanduva,SP
50304,05c59c1adfa648942fe0966db6296ff1,a8890e03c9673f8646efd590b21ebf6c,1526,sao paulo,SP
23177,d014e48c9b9e60f7804f0a43d4891b30,f99725ab98ff67b04beabcdbff645ee7,95370,barracao,RS


### EXPORT: tutti i dataframe puliti nella cartella output

In [46]:
import os

# Crea la cartella output se non esiste
os.makedirs("output", exist_ok=True)

dataframes_to_export = {
    "list_product":        df_list_product,
    "orders_items":        df_orders_items,
    "order_dataset":       df_order_dataset,
    "order_review":        df_order_review,
    "order_payments":      df_order_payments,
    "geolocation_dataset": df_geolocation_dataset,
    "list_seller":         df_list_seller,
    "customers":           df_customers,
}

for name, df in dataframes_to_export.items():
    df.to_csv(f"output/{name}.csv", index=False)